In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
import pandas as pd
import csv
import time

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--incognito')
service = Service("/Users/tumendemberelshalkhaan/Downloads/chromedriver")
driver = webdriver.Chrome(service=service, options = chrome_options)
driver.get('https://nysdoccslookup.doccs.ny.gov/') 

In [ ]:
ny = pd.DataFrame(columns=[])

In [1]:
import csv

with open('/Users/tumendemberelshalkhaan/Desktop/NY_DOC.csv', newline='') as csvfile:
    reader = csv.reader(csvfile)
    ids = [item for row in reader for item in row]

In [ ]:
driver.find_element("id", "lst").send_keys('a')
driver.find_element("xpath", "//button[contains(text(), 'Search')]").click() 

time.sleep(2)   
while True:
    try:
        next = driver.find_element("xpath", "//a[contains(text(), 'Next')]")
    except:
        break
    
    table = driver.find_element('xpath', "//table")
    rows = table.find_elements('xpath', '//tr')
    
    for row in rows:

        cols = row.find_elements('xpath', "//td")
        link = cols[0]
        id = link.text
        link.click()
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'lxml')
        status = soup.find('h5', text = 'Custody Status:').findParent().find_next_sibling('div').get_text()

        if 'IN CUSTODY' in status:

            name = soup.find('h3', class_ = 'font-weight-bold').get_text()
            demos = soup.find_all('div', class_ = 'col-sm-3')
            race = demos[5].get_text()
            dob = demos[6].get_text()
            age = demos[7].get_text()

            county = soup.find('h5', text = 'County of Commitment:').findParent().find_next_sibling('div').get_text()
            fac = soup.find('h5', text = 'Housing / Releasing Facility:').findParent().find_next_sibling('div').get_text()
            min_sen = soup.find('h5', text = 'Aggregate Minimum Sentence').findParent().find_next_sibling('div').get_text()
            max_sen = soup.find('h5', text = 'Aggregate Maximum Sentence').findParent().find_next_sibling('div').get_text()
            admit_date = soup.find('h5', text = 'Date Received (current):').findParent().find_next_sibling('div').get_text()

            table = soup.find('table', class_ = 'table table-bordered text-nowrap').find('tbody')
            offenses = []
            for row in table.find_all('tr'):
                data = [cell.get_text(strip=True) for cell in row.find_all('td')]
                if data[0]:
                    offenses.append(data[0])

            rows = []           

            for offense in offenses:
                row = {
                    'ID': id,
                    'Name': name,
                    'Date Of Birth': dob,
                    'Race': race,
                    'Facility': fac,
                    'Offense Description': offense,
                    'Admission Date': admit_date,
                    'Minimum Sentence': min_sen,
                    'Maximum Sentence': max_sen,
                    'County': county
                }
                rows.append(row)

            ny = pd.concat([ny, pd.DataFrame(rows)])
        time.sleep(1)
        driver.find_element('xpath', "//a[contains(text(), '< Back to Search Results')]").click()
    
        
    try:
        next.click()
        
    except:
        break

In [ ]:
ny.to_csv('/Users/tumendemberelshalkhaan/Desktop/New_York_Raw.csv', index=False)